## 9. Disk stability

We will create an exponential disk that is isothermal in the vertical direction, in a background NFW potential, and let it run. Solving for an equilibrium disk configuration is very hard, and so ``IC.expdisk`` does an approximate solution.

First we need to define the analytic NFW potential. This is both used in the simulation, so all particles will feel this extra force in addition to the force from all the other particles, and is also required by the IC code so that it knows what rotation velocities to give the particles.

In [ ]:
# Galpy NFW potential with a scale mass of 2e11 Msun and scale radius of 20 kpc
NFWpot = potential.NFWPotential(amp=2e11*u.Msun, a=20*u.kpc)

In [ ]:
# Create an exponential disk with central surface brightness of 100 Msun/pc^2, exponential scale length of 2kpc
# and vertical height of 0.2 kpc, a radial velocity dispersion scale of 20 km/s, and
# using the NFWpot rotation curve as an external force with 5000 particles
expdisk_IC = IC.expdisk(N=5000, sigma0=100*u.Msun/u.pc**2, Rd=2*u.kpc, z0=0.2*u.kpc, sigmaR_Rd=20*u.km/u.s,\
                       external_rotcurve=NFWpot.vcirc)

In [ ]:
# Create a simulation with a 2.5 Myr time step and 0.1 kpc softening
sim_disk = Simulation(dt=2.5*u.Myr, eps=0.1*u.kpc)
# Add the disk IC
sim_disk.add_IC(expdisk_IC)
# Add the NFW potential as an external force
sim_disk.add_external_force(NFWpot)

In [ ]:
# Run for 400 time steps
sim_disk.run(400)

Let's make a movie of the particle positions and velocities, both face-on and edge-on.

In [ ]:
fig = plt.figure(figsize=(15,15))

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax_xy = fig.add_subplot(221, aspect=1.0)
    ax_xz = fig.add_subplot(223, aspect=1.0)
    ax_vxy = fig.add_subplot(222, aspect=1.0)
    ax_vxz = fig.add_subplot(224, aspect=1.0)
    
    # First plot: particle positions xy in units of kpc
    plot_xy = sim_disk.plot_particles(snap=frame, parm='pos', coords='xy', unit=u.kpc, \
                                 xlim=[-20,20], ylim=[-20,20], ax=ax_xy)
    # Second plot: velocity in xy in units of km/s
    plot_vxy = sim_disk.plot_particles(snap=frame, parm='vel', coords='xy', unit=u.km/u.s, \
                                xlim=[-200,200], ylim=[-200,200], ax=ax_vxy)

    # Same but xz
    plot_xz = sim_disk.plot_particles(snap=frame, parm='pos', coords='xz', unit=u.kpc, \
                                 xlim=[-20,20], ylim=[-20,20], ax=ax_xz)
    plot_vxz = sim_disk.plot_particles(snap=frame, parm='vel', coords='xz', unit=u.km/u.s, \
                                xlim=[-200,200], ylim=[-200,200], ax=ax_vxz)
        
    return plot_xy, plot_vxy, plot_xz, plot_vxz

ms_per_frame = 40   # 25 frames per second
anim = FuncAnimation(fig, animate, frames=sim_disk.Nsnap, interval=ms_per_frame)
anim.save('disk_posvel.mp4')

plt.close(fig)

Think about what you're seeing, both initially and after it settles down. Does it reach a reasonably steady state, or does it continue to evolve?

Let's look at the evolution of the surface density and vertical density profiles.

In [ ]:
fig = plt.figure(figsize=(18,9))

plotz_max = 2.

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax_surfdens = fig.add_subplot(121)
    ax_vertdens = fig.add_subplot(122)
    
    # Create a pynbody snapshot of the given snapshot and make a 3D profile from it
    snap = sim_disk.pyn_snap(frame)
    # Make a 2D profile of the snapshot (so we get surface density as a function of cylindrical radius)
    prof = profile.Profile(snap, ndim=3, nbins=200)
    # Vertical profile that gives us density as a function of height. Use all radii between 0 and 25 kpc.
    zprof = profile.VerticalProfile(snap, 0., 25., plotz_max)
    
    # Surface density profile
    densplot = ax_surfdens.plot(prof['rbins'], prof['density'])
    ax_surfdens.set_xscale('log')
    ax_surfdens.set_yscale('log')
    ax_surfdens.set_xlim(0.1, 20)
    ax_surfdens.set_ylim(1e2,1e8)
    radial_label = 'r (${0}$)'.format(prof['rbins'].units.latex())
    ax_surfdens.set_xlabel(radial_label)
    ax_surfdens.set_ylabel('$\\Sigma$ (${0}$)'.format(prof['density'].units.latex()))
    ax_surfdens.set_title('{0:.1f}'.format(sim_disk.times[frame]))
    
    # Vertical density profile
    vertplot = ax_vertdens.plot(zprof['rbins'], zprof['density'].in_units('Msol pc^-3'))
    ax_vertdens.set_yscale('log')
    ax_vertdens.set_xlim(0,plotz_max)
    ax_vertdens.set_ylim(3e-6, 3e-3)
    vertical_axis_label = 'z (${0}$)'.format(zprof['rbins'].units.latex())
    ax_vertdens.set_xlabel(vertical_axis_label)
    ax_vertdens.set_ylabel('$\\rho$ [M$_{\odot}$ pc$^{-3}$]')
        
    return densplot, vertplot

anim = FuncAnimation(fig, animate, frames=sim_disk.Nsnap, interval=ms_per_frame)
anim.save('disk_profiles.mp4')

plt.close(fig)

Look at these. How do they connect to how we tried to set up the disk? How do they connect to what you see in the particle movies?

We can also use profiles to make movies of the 3 velocity components. We'll plot both the mean velocities and the dispersions.

In [ ]:
fig = plt.figure(figsize=(18,18))

radius_range = [0,20]
vrange = [-15,40]

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax_vr = fig.add_subplot(221)
    ax_vphi = fig.add_subplot(222)
    ax_vz = fig.add_subplot(223)
    
    # Create a pynbody snapshot of the given snapshot and make a 3D profile from it
    snap = sim_disk.pyn_snap(frame)
    # Make a 2D profile of the snapshot
    prof = profile.Profile(snap, ndim=3, nbins=200)
    radial_label = 'r (${0}$)'.format(prof['rbins'].units.latex())

    vrplot = ax_vr.plot(prof['rbins'], prof['vr'], label='$<v_R>$')
    ax_vr.plot(prof['rbins'], prof['vr_disp'], label='$\sigma_R$')
    ax_vr.plot(radius_range, [0,0], color='gray')
    ax_vr.set_xlim(*radius_range)
    ax_vr.set_ylim(*vrange)
    ax_vr.set_xlabel(radial_label)
    ax_vr.set_ylabel('Radial Velocity')
    ax_vr.legend(loc='upper right')

    vphiplot = ax_vphi.plot(prof['rbins'], prof['vphi'], label='$<v_{\phi}>$')
    ax_vphi.plot(prof['rbins'], prof['vphi_disp'], label='$\sigma_{\phi}$')
    ax_vphi.set_xlim(*radius_range)
    ax_vphi.set_ylim(0,100)  # These will be higher
    ax_vphi.set_xlabel(radial_label)
    ax_vphi.set_ylabel('Azimuthal Velocity')
    ax_vphi.legend(loc='upper left')
    
    vzplot = ax_vz.plot(prof['rbins'], prof['vz'], label='$<v_z>$')
    ax_vz.plot(prof['rbins'], prof['vz_disp'], label='$\sigma_z$')
    ax_vz.plot(radius_range, [0,0], color='gray')
    ax_vz.set_xlim(*radius_range)
    ax_vz.set_ylim(*vrange)
    ax_vz.set_xlabel(radial_label)
    ax_vz.set_ylabel('Vertical Velocity')
    ax_vz.legend(loc='upper right')
    
    return vrplot, vphiplot, vzplot

anim = FuncAnimation(fig, animate, frames=sim_disk.Nsnap, interval=ms_per_frame)
anim.save('disk_vprofs.mp4')

plt.close(fig)

Think about what you're seeing, both in terms of the mean velocities and their dispersions. How does this relate to how you see the densities evolve?

## 10. Play with the disk

**FIXME:** Take each of the disk parameters and try changing it to see what effect it has. Do the equilibrium configurations (if they exist) reflect these changes in the way you would expect? Do the presence or absence of disk instabilities change in the way you would expect?

## 11. Find a particle and trace its orbit

Let's find a particle with particular properties and trace its orbit. We'll choose a particle based on its original position and velocity. Pick a position and velocity for a particle that you would expect to be on a mostly (but not perfectly) circular orbit, i.e. one that should have a slight orbital eccentricity in the orbital plane and a slight vertical bobbing.

**FIXME:** Put your preferred particle position and velocity here.

In [ ]:
R_desired = #FIXME
z_desired = #FIXME
vR_desired = #FIXME
vz_desired = #FIXME

In [ ]:
# Compare all particles to this, relative to a "typical" distance and velocity scale
Rscale = 2*u.kpc
zscale = 0.2*u.kpc
vscale = 2*u.km/u.s
disk_R = np.sqrt(sim_disk.positions[0,:,0]**2 + sim_disk.positions[0,:,1]**2)
disk_z = sim_disk.positions[0,:,2]
disk_vR = (sim_disk.velocities[0,:,0]*sim_disk.positions[0,:,0] + sim_disk.velocities[0,:,1]*sim_disk.positions[0,:,1])/disk_R
disk_vz = sim_disk.velocities[0,:,2]
how_dissimilar = ((disk_R - R_desired)/Rscale)**2 + ((disk_z - z_desired)/zscale)**2 + \
    ((disk_vR - vR_desired)/vscale)**2 + ((disk_vz - vz_desired)/vscale)**2

In [ ]:
# np.argmin tells you which element in an array has the smallest value
best_particle = np.argmin(how_dissimilar)

**FIXME:** Check that it did an okay job using this. If not, go back and adjust your desired parameters. You won't find a perfect match, but make sure that it is "mostly but not perfectly circular".

In [ ]:
print('R: ',disk_R[best_particle])
print('z: ',disk_z[best_particle])
print('vR: ',disk_vR[best_particle])
print('vz: ',disk_vz[best_particle])

Now let's plot this particle's orbit.

**FIXME:** Plot the xy and xz trajectories of ``best_particle``.

In [ ]:
# xy and xz projections
fig = plt.figure(figsize=(10,4))
ax_xy = fig.add_subplot(121, aspect=1.0)
ax_xz = fig.add_subplot(122, aspect=1.0)
ax_xy.plot(#FIXME
ax_xz.plot(#FIXME
# Label appropriately
ax_xy.set_#FIXME...
ax_xz.set_#FIXME...

And let's plot how $R$ and $z$ evolve with time.

Note: You might want to use the ``%matplotlib notebook`` pragma to do the analysis of this.

In [ ]:
fig = plt.figure(figsize=(12,4))
ax_R = fig.add_subplot(121)
ax_z = fig.add_subplot(122)
# Define R of best_particle for convenience
best_R = np.sqrt(sim_disk.positions[:,best_particle,0]**2 + sim_disk.positions[:,best_particle,1]**2)
# time label
timelabel = 't ({0})'.format(sim_disk.times.unit)

ax_R.plot(sim_disk.times, best_R)
ax_R.set_xlabel(timelabel)
ax_R.set_ylabel('R ({0})'.format(best_R.unit))

ax_z.plot(sim_disk.times, sim_disk.positions[:,best_particle,2])
ax_z.set_xlabel(timelabel)
ax_z.set_ylabel('z ({0})'.format(sim_disk.positions.unit))

How do these orbits compare to the orbits we computed in Experiment 3? What is your best estimate for the epicyclic frequency, based on this plot? How about the vertical frequency? Are they constant? Is there any relation between the vertical frequency and $R$? What would you expect the frequencies to be at early times based on the NFW + disk potential with the surface density profile we gave and the radius you chose? Do they agree with what you measure from the plot?

## 12. Do something interesting

You know the drill... try something else! For example, you could:
 - Take some parameters to ridiculous extremes and see whether it reacts how you expect.
 - Add an N-body bulge (e.g. as a Hernquist IC).
     - If you do this, make sure to include its effect on the rotation curve when you set up the disk.
 - Use a different potential for the halo.
 - Add a time-dependent potential like a bar (note: if you do this, you will need to use the ``add_external_timedependent_force`` instead of ``add_external_force``).
 - Run for a very long time to see the long term evolution of the disk.
 - Try using a halo potential that is different from the rotation curve used to set up the disk.